In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col 
from linearmodels import RandomEffects
from linearmodels import PanelOLS
import scipy as sp

## Problem 1

In [3]:
df = pd.read_stata("jtrain.dta")
df.head()

,year,fcode,employ,sales,avgsal,scrap,rework,tothrs,union,grant,...,grant_1,clscrap,cgrant,clemploy,clsales,lavgsal,clavgsal,cgrant_1,chrsemp,clhrsemp
0,1987,410032.0,100.0,47000000.0,35000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,10.463103,NaN,NaN,NaN,NaN
1,1988,410032.0,131.0,43000000.0,37000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,0.270027,-0.088949,10.518673,0.055570,0.0,-8.946565,-1.165385
2,1989,410032.0,123.0,49000000.0,39000.0,NaN,NaN,8.0,0,0,...,0,NaN,0,-0.063013,0.130621,10.571317,0.052644,0.0,0.198597,0.047832
3,1987,410440.0,12.0,1560000.0,10500.0,NaN,NaN,12.0,0,0,...,0,NaN,0,NaN,NaN,9.259130,NaN,NaN,NaN,NaN
4,1988,410440.0,13.0,1970000.0,11000.0,NaN,NaN,12.0,0,0,...,0,NaN,0,0.080043,0.233347,9.305651,0.046520,0.0,0.000000,0.000000


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471 entries, 0 to 470
Data columns (total 30 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   year      471 non-null    int16  
 1   fcode     471 non-null    float32
 2   employ    440 non-null    float64
 3   sales     373 non-null    float32
 4   avgsal    406 non-null    float32
 5   scrap     162 non-null    float32
 6   rework    123 non-null    float32
 7   tothrs    415 non-null    float64
 8   union     471 non-null    int8   
 9   grant     471 non-null    int8   
 10  d89       471 non-null    int8   
 11  d88       471 non-null    int8   
 12  totrain   465 non-null    float64
 13  hrsemp    390 non-null    float32
 14  lscrap    162 non-null    float32
 15  lemploy   440 non-null    float32
 16  lsales    373 non-null    float32
 17  lrework   121 non-null    float32
 18  lhrsemp   390 non-null    float32
 19  lscrap_1  108 non-null    float32
 20  grant_1   471 non-null    int8  

## (a)

In [5]:
df1 = df[["fcode", "hrsemp", "d88", "d89", "lemploy", "grant", "grant_1"]]
df1 = df1.dropna()

## Drop the firms if they don't have data for all three time periods 
df1 = df1.groupby("fcode").filter(lambda x: len(x)==3)
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 0 to 470
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   fcode    372 non-null    float32
 1   hrsemp   372 non-null    float32
 2   d88      372 non-null    int8   
 3   d89      372 non-null    int8   
 4   lemploy  372 non-null    float32
 5   grant    372 non-null    int8   
 6   grant_1  372 non-null    int8   
dtypes: float32(3), int8(4)
memory usage: 8.7 KB


In [6]:
## Compute the first difference
df1_diff = df1.groupby("fcode").diff(1).dropna()
df1_diff["const"] = 1

In [7]:
## Run the first-difference OLS regression

exog_var = ["const", "d89", "lemploy", "grant", "grant_1"]

result1 = sm.OLS(df1_diff["hrsemp"], df1_diff[exog_var]).fit() 
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                 hrsemp   R-squared:                       0.476
Model:                            OLS   Adj. R-squared:                  0.467
Method:                 Least Squares   F-statistic:                     55.17
Date:                Sun, 27 Mar 2022   Prob (F-statistic):           4.75e-33
Time:                        17:12:07   Log-Likelihood:                -1086.1
No. Observations:                 248   AIC:                             2182.
Df Residuals:                     243   BIC:                             2200.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5910      1.956     -0.302      0.7

### [ANS] 

There are 124 firms used in the estimation. We use 372 observations for all three time periods.

## (b)

 The coefficient of $Grant_{-1}$ is $32.4834$ and its p-value us $0.000$, which shows that it is significant.
Intuitively, the more grants firms receive, the more they train their employees. 

## (c)

Yes. Practically, we may believe that the job training grants can have a long-term effect on the job training per employee. 
However, it cannot be supported by the dataset.

## (d)

Large firms trains their employees more.  On average, one percent increase in employees leads to $1.0706$ increases in hours per employee. 
Notice that this coefficient is insignificant.

## Problem 2

## (a)

In [8]:
df = pd.read_stata("murder.dta")
df.head()

,id,state,year,mrdrte,exec,unem,d90,d93,cmrdrte,cexec,cunem,cexec_1,cunem_1
0,1,AL,87,9.3,2,7.8,0,0,NaN,NaN,NaN,NaN,NaN
1,1,AL,90,11.6,5,6.8,1,0,2.3,3.0,-1.0,NaN,NaN
2,1,AL,93,11.6,2,7.5,0,1,0.0,-3.0,0.7,3.0,-1.0
3,2,AK,87,10.1,0,10.8,0,0,NaN,NaN,NaN,NaN,NaN
4,2,AK,90,7.5,0,6.9,1,0,-2.6,0.0,-3.9,NaN,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 152
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       153 non-null    int8   
 1   state    153 non-null    object 
 2   year     153 non-null    int8   
 3   mrdrte   153 non-null    float32
 4   exec     153 non-null    int8   
 5   unem     153 non-null    float32
 6   d90      153 non-null    int8   
 7   d93      153 non-null    int8   
 8   cmrdrte  102 non-null    float32
 9   cexec    102 non-null    float64
 10  cunem    102 non-null    float32
 11  cexec_1  51 non-null     float64
 12  cunem_1  51 non-null     float32
dtypes: float32(5), float64(2), int8(5), object(1)
memory usage: 8.5+ KB


In [10]:
df2 = df[["id", "mrdrte", "d90", "d93", "exec", "unem"]]
df2 = df2.dropna()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153 entries, 0 to 152
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      153 non-null    int8   
 1   mrdrte  153 non-null    float32
 2   d90     153 non-null    int8   
 3   d93     153 non-null    int8   
 4   exec    153 non-null    int8   
 5   unem    153 non-null    float32
dtypes: float32(2), int8(4)
memory usage: 3.0 KB


In [11]:
df2["const"] = 1

## Run Pooled OLS regression
result2 = sm.OLS(df2["mrdrte"],
                   df2[["const", "d90", "d93", "exec", "unem"]]).fit() 
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     3.047
Date:                Sun, 27 Mar 2022   Prob (F-statistic):             0.0190
Time:                        17:12:08   Log-Likelihood:                -549.96
No. Observations:                 153   AIC:                             1110.
Df Residuals:                     148   BIC:                             1125.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.8644      3.070     -0.607      0.5

## (b)

In [12]:
## Compute the first difference
df2_diff = df2.groupby("id").diff(1).dropna()
df2_diff["const"] = 1
df2_diff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 1 to 152
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   mrdrte  102 non-null    float32
 1   d90     102 non-null    float32
 2   d93     102 non-null    float32
 3   exec    102 non-null    float32
 4   unem    102 non-null    float32
 5   const   102 non-null    int64  
dtypes: float32(5), int64(1)
memory usage: 3.6 KB


In [13]:
## Run the first-difference OLS regression

result2 = sm.OLS(df2_diff["mrdrte"],
                   df2_diff[["const", "d93", "exec", "unem"]]).fit() 
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                 mrdrte   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8429
Date:                Sun, 27 Mar 2022   Prob (F-statistic):              0.474
Time:                        17:12:08   Log-Likelihood:                -291.48
No. Observations:                 102   AIC:                             591.0
Df Residuals:                      98   BIC:                             601.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5110      0.661      2.286      0.0

## (c)

Yes. The coefficient of $UNEM_{it}$ becomes insignificant under first-difference estimation.

If we don't partial out the individual component $c_i$ from the estimation, we may overestimate the effect of unemployment. 

## (d)

If $\epsilon_{it-1}$, the shock last year, affects the excutions this year, then $EXEC_{it}$ would not be strictly exogenous anymore. Such a case may happen because the policy maker may tend to support more excutions when the number of murders gets higher.

## Problem 3

## (a)

In [14]:
df = pd.read_stata("cornwell.dta")
df.head()

,county,year,crmrte,prbarr,prbconv,prbpris,avgsen,polpc,density,taxpc,...,lpctymle,lpctmin,clcrmrte,clprbarr,clprbcon,clprbpri,clavgsen,clpolpc,cltaxpc,clmix
0,1,81,0.039885,0.289696,0.402062,0.472222,5.61,0.001787,2.307159,25.697630,...,-2.433870,3.006608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,82,0.038345,0.338111,0.433005,0.506993,5.59,0.001767,2.330254,24.874252,...,-2.449038,3.006608,-0.039376,0.154542,0.074143,0.071048,-0.003571,-0.011364,-0.032565,0.030857
2,1,83,0.030305,0.330449,0.525703,0.479705,5.80,0.001836,2.341801,26.451443,...,-2.464036,3.006608,-0.235316,-0.022922,0.193987,-0.055326,0.036879,0.038413,0.061477,-0.244732
3,1,84,0.034726,0.362525,0.604706,0.520104,6.89,0.001886,2.346420,26.842348,...,-2.478925,3.006608,0.136180,0.092641,0.140006,0.080857,0.172213,0.026930,0.014670,-0.027331
4,1,85,0.036573,0.325395,0.578723,0.497059,6.55,0.001924,2.364896,28.140337,...,-2.497306,3.006608,0.051825,-0.108054,-0.043918,-0.045320,-0.050606,0.020199,0.047223,0.172125


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 630 entries, 0 to 629
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   county    630 non-null    int16  
 1   year      630 non-null    int8   
 2   crmrte    630 non-null    float32
 3   prbarr    630 non-null    float32
 4   prbconv   630 non-null    float32
 5   prbpris   630 non-null    float32
 6   avgsen    630 non-null    float32
 7   polpc     630 non-null    float32
 8   density   630 non-null    float32
 9   taxpc     630 non-null    float32
 10  west      630 non-null    int8   
 11  central   630 non-null    int8   
 12  urban     630 non-null    int8   
 13  pctmin80  630 non-null    float32
 14  wcon      630 non-null    float32
 15  wtuc      630 non-null    float32
 16  wtrd      630 non-null    float32
 17  wfir      630 non-null    float32
 18  wser      630 non-null    float32
 19  wmfg      630 non-null    float32
 20  wfed      630 non-null    float3

In [16]:
year = pd.Categorical(df.year)
df3 = df.set_index(['county', 'year'])
df3['year'] = year
df3['const'] = 1

In [17]:
## Estimate the model by random effects

exog_var = ['const', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87',
            'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']
exog = df3[exog_var]
model_RE = RandomEffects(df3.lcrmrte, exog)
result_RE = model_RE.fit()
print(result_RE)

                        RandomEffects Estimation Summary                        
Dep. Variable:                lcrmrte   R-squared:                        0.4264
Estimator:              RandomEffects   R-squared (Between):              0.4203
No. Observations:                 630   R-squared (Within):               0.4281
Date:                Sun, Mar 27 2022   R-squared (Overall):              0.4210
Time:                        17:12:08   Log-likelihood                    321.61
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      41.756
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,618)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             41.756
                            

In [18]:
## Estimate the model by fixed effects

model_FE = PanelOLS(df3.lcrmrte, exog, entity_effects=True)
result_FE = model_FE.fit()
print(result_FE)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lcrmrte   R-squared:                        0.4342
Estimator:                   PanelOLS   R-squared (Between):              0.3746
No. Observations:                 630   R-squared (Within):               0.4342
Date:                Sun, Mar 27 2022   R-squared (Overall):              0.3798
Time:                        17:12:08   Log-likelihood                    405.58
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      36.911
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(11,529)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             36.911
                            

## (b)

In [19]:
## Compute W_i by averaging all explanatory variables across time

time_average = df3.groupby("county").mean()

## Merge the dataset

df3_merged = df3.merge(time_average, how='left', 
                       left_index=True, right_index=True, suffixes=('', '_avg'))

In [20]:
## Estimate the model by random effects

exog_var = ['const', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87',
            'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
            'lprbarr_avg', 'lprbconv_avg', 'lprbpris_avg', 'lavgsen_avg', 'lpolpc_avg']
exog = df3_merged[exog_var]
model_RE2 = RandomEffects(df3_merged.lcrmrte, exog)
result_RE2 = model_RE2.fit()
print(result_RE2)

                        RandomEffects Estimation Summary                        
Dep. Variable:                lcrmrte   R-squared:                        0.4940
Estimator:              RandomEffects   R-squared (Between):              0.7099
No. Observations:                 630   R-squared (Within):               0.4342
Date:                Sun, Mar 27 2022   R-squared (Overall):              0.6859
Time:                        17:12:08   Log-likelihood                    363.74
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      37.410
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(16,613)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             37.410
                            

In [21]:
## Do the Wald-Test for the coefficients

formula = 'lprbarr_avg = lprbconv_avg = lprbpris_avg  = lavgsen_avg = lpolpc_avg = 0'
result_RE2.wald_test(formula=formula)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 81.9424
P-value: 0.0000
Distributed: chi2(5)
WaldTestStatistic, id: 0x7fc5808e1070

## [ANS] 

It shows that we can reject the null hypothesis under $0.01$ significant level.

## (c)

In [22]:
## Add logarithms of the nine wage variables

exog_var = ['const', 'd82', 'd83', 'd84', 'd85', 'd86', 'd87',
            'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc',
            'lwcon', 'lwtuc', 'lwtrd', 'lwfir', 'lwser', 'lwmfg', 'lwfed', 'lwsta', 'lwloc']
exog = df3[exog_var]

## Estimate the model by fixed effects

model_FE2 = PanelOLS(df3.lcrmrte, exog, entity_effects=True)
result_FE2 = model_FE2.fit()
print(result_FE2)

                          PanelOLS Estimation Summary                           
Dep. Variable:                lcrmrte   R-squared:                        0.4575
Estimator:                   PanelOLS   R-squared (Between):              0.2458
No. Observations:                 630   R-squared (Within):               0.4575
Date:                Sun, Mar 27 2022   R-squared (Overall):              0.2642
Time:                        17:12:08   Log-likelihood                    418.79
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      21.923
Entities:                          90   P-value                           0.0000
Avg Obs:                       7.0000   Distribution:                  F(20,520)
Min Obs:                       7.0000                                           
Max Obs:                       7.0000   F-statistic (robust):             21.923
                            

In [23]:
## Do the Wald-Test for the coefficients

formula = 'lwcon = lwtuc = lwtrd = lwfir = lwser = lwmfg = lwfed = lwsta = lwloc = 0'
result_FE2.wald_test(formula=formula)

Linear Equality Hypothesis Test
H0: Linear equality constraint is valid
Statistic: 22.2649
P-value: 0.0081
Distributed: chi2(9)
WaldTestStatistic, id: 0x7fc5b07a6910

## [ANS] 

We can reject the null hypothesis that the coefficient of these variables are equal to zero under $0.01$ significant level.

In [24]:
df3_diff = df3.groupby("county").diff(1)
df3_diff.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 630 entries, (1, 81) to (197, 87)
Data columns (total 59 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   crmrte    540 non-null    float32
 1   prbarr    540 non-null    float32
 2   prbconv   540 non-null    float32
 3   prbpris   540 non-null    float32
 4   avgsen    540 non-null    float32
 5   polpc     540 non-null    float32
 6   density   540 non-null    float32
 7   taxpc     540 non-null    float32
 8   west      540 non-null    float32
 9   central   540 non-null    float32
 10  urban     540 non-null    float32
 11  pctmin80  540 non-null    float32
 12  wcon      540 non-null    float32
 13  wtuc      540 non-null    float32
 14  wtrd      540 non-null    float32
 15  wfir      540 non-null    float32
 16  wser      540 non-null    float32
 17  wmfg      540 non-null    float32
 18  wfed      540 non-null    float32
 19  wsta      540 non-null    float32
 20  wloc      540 non-nu

/Users/jhongyihuang/opt/anaconda3/lib/python3.9/site-packages/pandas/core/groupby/groupby.py:937: FutureWarning: dtype lost in 'diff()'. In the future this will raise a TypeError. Convert to a suitable dtype prior to calling 'diff'.
  return f(x, *args, **kwargs)


## (d)

In [25]:
## Compute the first difference

df3_diff = df3.groupby("county").diff(1)

df3_diff['const'] = 1 
exog_var = ['const', 'd83', 'd84', 'd85', 'd86', 'd87',
            'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']
df3_diff = df3_diff[["lcrmrte", 'const', 'd83', 'd84', 'd85', 'd86', 'd87',
            'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']].dropna()


## Estimate the model by first difference

result_diff = sm.OLS(df3_diff["lcrmrte"], exog=df3_diff[exog_var]).fit() 
print(result_diff.summary())

                            OLS Regression Results                            
Dep. Variable:                lcrmrte   R-squared:                       0.433
Model:                            OLS   Adj. R-squared:                  0.422
Method:                 Least Squares   F-statistic:                     40.32
Date:                Sun, 27 Mar 2022   Prob (F-statistic):           6.30e-59
Time:                        17:12:08   Log-Likelihood:                 248.48
No. Observations:                 540   AIC:                            -475.0
Df Residuals:                     529   BIC:                            -427.7
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0077      0.017      0.452      0.6

## [ANS] 

Compared with the fixed effect and random effect regression, the first-difference model has a smaller coefficient for $lnPOLPC_{it}$. This may reflect that the first difference reduces the serial correlation in the error terms. 

## (e)

No. The differences in the standard errors between FE and FD are quite small.

## (f)

In [28]:
## Extract the residual and run the AR(1) regression

resid = result_diff.resid
resid_lag = resid.groupby("county").shift(1)
result_AR_test = sm.OLS(resid, resid_lag, missing = 'drop').fit() 
print(result_AR_test.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.049
Model:                            OLS   Adj. R-squared (uncentered):              0.047
Method:                 Least Squares   F-statistic:                              22.97
Date:                Sun, 27 Mar 2022   Prob (F-statistic):                    2.24e-06
Time:                        17:12:46   Log-Likelihood:                          205.85
No. Observations:                 450   AIC:                                     -409.7
Df Residuals:                     449   BIC:                                     -405.6
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

## [ANS] 

We can reject the null hypothesis that the AR(1) coefficient is zero from the above table.